<a href="https://colab.research.google.com/github/iatra/COSC2673-Assignment2-Group9/blob/main/s4005248_s3980067.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning COSC2673-Assignment2-Group9
Author : Joelle Lee Ming Yin (s3980067) & Isabella Tranquilino (s4005248)

# Load Dataset

In [ ]:
from google.colab import files
uploaded = files.upload()  # This will prompt you to upload a file from your computer
